In [133]:
import requests
import pandas as pd
import os

In [134]:
# Get the current working directory
cwd = os.getcwd()
print(f"Current Working Directory: {cwd}")

# Define the relative path to states.csv
data_path = os.path.join(cwd, '..', 'data')

# Normalize path to avoid issues
data_path = os.path.abspath(data_path)
print(f"Resolved Data Path: {data_path}")

Current Working Directory: C:\Users\josue\Documents\Python\projects\Analyzing_US_Census_Data_in_Python\notebooks
Resolved Data Path: C:\Users\josue\Documents\Python\projects\Analyzing_US_Census_Data_in_Python\data


In [135]:
# API endpoint
HOST = "https://api.census.gov/data"
year = "2012"  # Adjust the year if necessary
dataset = "acs/acs5"
base_url = "/".join([HOST, year, dataset])

# Specify the variables you want
get_vars = ["NAME"]  # Use state_code and county_code

# My api_key is already in place
# api_key = 'YOUR_API_KEY'  # Your API Key here

# Create dictionary of predicates
predicates = {
    "get": ",".join(get_vars),
    "for":"metropolitan statistical area/micropolitan statistical area:*", 
    "key": api_key  # Include your API key here
}

# Make the request
r = requests.get(base_url, params=predicates)

# Create user-friendly column names
col_names = ["name","msa"]
# Load JSON response into DataFrame
msa_names = pd.DataFrame(columns=col_names, data=r.json()[1:])

In [136]:
# Define the file path for the CSV file
csv_file = data_path + '\\cbsa2fipsxw.csv'

# Read the CSV file into a DataFrame
msa_cs = pd.read_csv(csv_file)

# Select relevant columns from the CSV data for further analysis
msa_cs = msa_cs[['cbsacode', 'countycountyequivalent', 'statename', 'fipscountycode', 'fipsstatecode']]

# Rename columns to more meaningful names for better readability
msa_cs.columns = ['msa', 'county_name', 'state_name', 'county', 'state']

# Convert numeric columns to strings with leading zeros to match required lengths
msa_cs['msa'] = msa_cs['msa'].astype('str').str.zfill(5)   # MSA code as a 5-digit string
msa_cs['county'] = msa_cs['county'].astype('str').str.zfill(3)  # County code as a 3-digit string
msa_cs['state'] = msa_cs['state'].astype('str').str.zfill(2)  # State code as a 2-digit string

# Merge the county-state DataFrame with the names 
# Use a left join to retain all data from msa_cs and match on 'msa'
msa_def = msa_cs.merge(msa_names, on='msa', how='left')

# Reorganize and rename columns for final output
# Select and reorder columns to have them in a desired format
msa_def = msa_def[['msa', 'name', 'county_name', 'state_name', 'state', 'county']]

# Rename columns for clarity in final DataFrame
msa_def.columns = ['msa', 'msa_name', 'county_name', 'state_name', 'state', 'county']
